In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# Define constants (We use atomic units for everything in this demo)
hbar = 1

# Define the interactive widgets

# Quantum number v can be any integer between 0 and 20
v_options = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description=r'$v$:',
    disabled=False,
)

# Max Quantum number vmax to be plotted, can be any integer between 0 and 20
vmax_options = widgets.IntSlider(
    value=4,
    min=0,
    max=20,
    step=1,
    description=r'$v_{max}$:',
    disabled=False,
)

# Angular frequency omega can be any float value between 0.01 and 10 (atomic unit) inputed by the user
omega_options = widgets.BoundedFloatText(
    description=r'$\omega$:',                             
    value=2.0,
    min=0.01,
    max=10.0,
    padding = 8)

# Mass m can be any float value between 0.1 and 10 (atomic unit) inputed by the user
m_options = widgets.BoundedFloatText(
    description=r'$m$:',                             
    value=0.5,
    min=0.01,
    max=10.0,
    padding = 8)

# Function that return hermite polynomial Hv(sqrt(alpha)x) for given m, omega,v
def Hv_func(alpha,v):
    
    # Set up the Hermite polynomial
    domain = [-1,1]
    window = [-np.sqrt(alpha),np.sqrt(alpha)]
    coeff = [0]*v
    coeff.append(1)
    return np.polynomial.hermite.Hermite(coeff,domain=domain, window=window)

In [3]:
fig,(ax1,ax2,ax3) = plt.subplots(3,sharex=True)
plt.subplots_adjust(hspace=0)

@widgets.interact(v=v_options,m=m_options,omega=omega_options)
def update(v,m,omega):
    ax1.cla()
    ax2.cla()
    ax3.cla()
    
    # Set the range of x values we want to plot
    npoints=2000
    xlower = -10
    xupper = 10
    
    # obtain the Hermite polynomial fuction
    alpha= m*omega/hbar    
    Hv = Hv_func(alpha,v)
    
    # obtain the Hermite polynomial curve data for the range of x values
    x,y=Hv.linspace(npoints,[xlower,xupper])
    
    
    # Add a title
    plt.sca(ax1)
    
    plt.title(r'Harmonic Oscillator'+ '\n'
              r'$m$= '+str(m) + ', $\omega$=' + str(omega) + 
              r', $\alpha$=' + str(alpha)+ r', $v$=' + str(v))
    
 
    #############################################
    # First work on the Hermite polynomial part #
    #############################################
    # Add y Label
    plt.ylabel(r'$H_v(\sqrt{\alpha}x)$') 
    
    # Plot
    plt.plot(x,y,c='b') 
    ymax = 2**(v/2.0)*np.sqrt(float(np.math.factorial(v)))/((alpha/np.pi)**0.25)
    ymin = -ymax
    plt.ylim(ymin,ymax)
    
    # Set grid
    plt.grid(True)
     
    
    #####################
    # Then work on Psi  #
    #####################
    y = 1/(2**(v/2.0)*np.sqrt(float(np.math.factorial(v))))*(alpha/np.pi)**0.25*y*np.exp(-1/2.0*alpha*x**2)
    
    plt.sca(ax2)

    # Add y Label
    plt.ylabel(r'$\psi(x)$')
    # Plot
    plt.plot(x,y,c='b')   
    
    # Set grid
    plt.grid(True)

    # Calculate energy of the state with quantum number v
    E = hbar*omega*(v+ 1/2.0)
    # Calculate the positive turning point xt_plus based on E
    xt_plus = np.sqrt(2.0/(m*omega**2)*E)
    xt_minus = -xt_plus  
    
    # Plot the Classical turning points
    plt.plot([xt_minus, xt_minus],[-1,1],c='k')
    plt.plot([xt_plus,xt_plus],[-1,1],c='k')
    #########################
    # Then  work on Psi**2  #
    #########################
    
    plt.sca(ax3)
    # Add X and y Label
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$|\psi(x)|^2$')
    
    # Plot
    plt.plot(x,y**2,c='r')
    plt.fill_between(x,y**2, color='r',alpha=0.5)
    plt.ylim(0,1.1)
    
    # Set grid
    plt.grid(True)
    
    # Plot the Classical turning points
    plt.plot([xt_minus, xt_minus],[-1,1],c='k')
    plt.plot([xt_plus,xt_plus],[-1,1],c='k')
    plt.text(xt_plus,0.5,'classical turning points')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='$v$:', max=20), BoundedFloatText(value=0.5, description=…

In [4]:
fig2=plt.figure()

@widgets.interact(vmax=vmax_options,m=m_options,omega=omega_options)
def update2(vmax,m,omega):
    fig2.clf()
    # Set the range of x values we want to plot
    npoints=2000
    xlower = -10
    xupper = 10


    # Let's first plot the parabola
    x = np.linspace(xlower,xupper,npoints)
    Vx = 0.5*m*omega**2*x**2
    plt.plot(x,Vx,c='k')
    plt.xlim(xlower,xupper)


    #Then plot the wave functions shifted in the y direction   
    alpha= m*omega/hbar  
    Emax = hbar*omega*(vmax+ 1/2.0)
    
    for v in range(0, vmax+1):
        # Calculate energy of the state with quantum number v
        E = hbar*omega*(v+ 1/2.0)
        # Calculate the positive turning point xt_plus based on E
        xt_plus = np.sqrt(2.0/(m*omega**2)*E)
        xt_minus = -xt_plus 
        # Plot the energy level line, intersecting with the parabola
        plt.plot([xt_minus, xt_plus],[E,E],c='k')


        # obtain the Hermite polynomial fuction
        Hv = Hv_func(alpha,v)
        # obtain the Hermite polynomial curve data for the range of x values
        # First obtain y as the Hermite polynomial
        x,y=Hv.linspace(npoints,[xlower,xupper])
        # Add the normalization coefficients and the gaussian funciton
        y = 1/(2**(v/2.0)*np.sqrt(float(np.math.factorial(v))))*(alpha/np.pi)**0.25*y*np.exp(-1/2.0*alpha*x**2)
        # Shift the wavefunction in y direction to align with the energy level
        y = y + E
        plt.plot(x,y,c='b')  
    plt.ylim(0,Emax+1)
    
    plt.title(r'Harmonic Oscillator $\psi(x)$ aligned with energy levels'+ '\n'
              r'$m$= '+str(m) + ', $\omega$=' + str(omega) + 
              r', $\alpha$=' + str(alpha)+ r', $v_{max}$=' + str(vmax))
    
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$E$(H$_a$)')







Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=4, description='$v_{max}$:', max=20), BoundedFloatText(value=0.5, descri…

In [5]:
fig3=plt.figure()

@widgets.interact(vmax=vmax_options,m=m_options,omega=omega_options)
def update3(vmax,m,omega):
    fig3.clf()
    # Set the range of x values we want to plot
    npoints=2000
    xlower = -10
    xupper = 10


    # Let's first plot the parabola
    x = np.linspace(xlower,xupper,npoints)
    Vx = 0.5*m*omega**2*x**2
    plt.plot(x,Vx,c='k')
    plt.xlim(xlower,xupper)


    #Then plot the wave functions shifted in the y direction   
    alpha= m*omega/hbar  
    Emax = hbar*omega*(vmax+ 1/2.0)
    
    for v in range(0, vmax+1):
        # Calculate energy of the state with quantum number v
        E = hbar*omega*(v+ 1/2.0)
        # Calculate the positive turning point xt_plus based on E
        xt_plus = np.sqrt(2.0/(m*omega**2)*E)
        xt_minus = -xt_plus 
        # Plot the energy level line, intersecting with the parabola
        plt.plot([xt_minus, xt_plus],[E,E],c='k')


        # obtain the Hermite polynomial fuction
        Hv = Hv_func(alpha,v)
        # obtain the Hermite polynomial curve data for the range of x values
        # First obtain y as the Hermite polynomial
        x,y=Hv.linspace(npoints,[xlower,xupper])
        # Add the normalization coefficients and the gaussian funciton
        y = 1/(2**(v/2.0)*np.sqrt(float(np.math.factorial(v))))*(alpha/np.pi)**0.25*y*np.exp(-1/2.0*alpha*x**2)
        # Get probability density
        y = y**2
        # Shift the probability density in y direction to align with the energy level
        y = y + E
        plt.plot(x,y,c='r')
        ybaseline = np.array([E]*npoints)
        plt.fill_between(x,y,ybaseline, color='r',alpha=0.5)       
    plt.ylim(0,Emax+1)
    
    plt.title(r'Harmonic Oscillator  $|\psi(x)|^2$ aligned with energy levels'+ '\n'
              r'$m$= '+str(m) + ', $\omega$=' + str(omega) + 
              r', $\alpha$=' + str(alpha)+ r', $v_{max}$=' + str(vmax))
    
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$E$(H$_a$)')






Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=4, description='$v_{max}$:', max=20), BoundedFloatText(value=0.5, descri…